In [19]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import gc  
import os
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

In [6]:
import os
print(os.getcwd())

C:\Users\Lisa\ST5188\submit


In [7]:
# defination the function to optimize the memory usage of the dataframe
def optimize_dataframe_for_memory(df):
 print("current memory usage: ", df.memory_usage().sum() / 1024**2, "MB")

 for col in df.columns:
  _max = df[col].max()
  _min = df[col].min()
  if df[col].dtype  == 'int64':
   if _min > np.iinfo(np.int8).min and _max < np.iinfo(np.int8).max:
    df[col] = df[col].astype(np.int8)
   elif _min > np.iinfo(np.int16).min and _max < np.iinfo(np.int16).max:
    df[col] = df[col].astype(np.int16)
   elif _min > np.iinfo(np.int32).min and _max < np.iinfo(np.int32).max:
    df[col] = df[col].astype(np.int32)
  elif df[col].dtype == 'float64':
   if _min > np.finfo(np.float16).min and _max < np.finfo(np.float16).max:
    df[col] = df[col].astype(np.float16)
   elif _min > np.finfo(np.float32).min and _max < np.finfo(np.float32).max:
    df[col] = df[col].astype(np.float32)
  else:
   continue
 
 print("new memory usage: ", df.memory_usage().sum() / 1024**2, "MB")
 return df

In [8]:
# BASE_PATH = "/kaggle/input/instacart/"
# order_products_train = optimize_dataframe_for_memory(pd.read_csv(os.path.join(BASE_PATH, "order_products__train.csv")))
# order_products_prior = optimize_dataframe_for_memory(pd.read_csv(os.path.join(BASE_PATH, "order_products__prior.csv")))
# orders = optimize_dataframe_for_memory(pd.read_csv(os.path.join(BASE_PATH, "orders.csv")))
# products = optimize_dataframe_for_memory(pd.read_csv(os.path.join(BASE_PATH, "products.csv")))

read_path = './data/'
save_path = './results/'
order_products_prior = optimize_dataframe_for_memory(pd.read_csv(read_path+"order_products__prior.csv", encoding='latin1'))
orders = optimize_dataframe_for_memory(pd.read_csv(read_path+"orders.csv"))
products = optimize_dataframe_for_memory(pd.read_csv(read_path+"products.csv"))
order_products_train = optimize_dataframe_for_memory(pd.read_csv(read_path+"order_products__train.csv"))

current memory usage:  989.8221778869629 MB
new memory usage:  340.25145626068115 MB
current memory usage:  182.70566940307617 MB
new memory usage:  68.51470470428467 MB
current memory usage:  1.5164833068847656 MB
new memory usage:  0.7109184265136719 MB
current memory usage:  42.25528335571289 MB
new memory usage:  13.204862594604492 MB


In [9]:
df_prior = pd.merge(order_products_prior, orders, on='order_id', how='inner')
df_prior = pd.merge(df_prior, products, on='product_id', how='left')

dict_agg5 = {
            #'add_to_cart_order': ['mean'],
            #'days_since_prior_order': ['mean'],
            'reordered': ['count']
            #'order_number': ['max']
            }
u_prod_f = df_prior.groupby(['user_id', 'product_id']).agg(dict_agg5)
u_prod_f.columns = ['u_p_orders_total']
u_prod_f.reset_index(inplace=True)
u_prod_f = pd.merge(u_prod_f, products[['product_id', 'product_name']], on='product_id', how='left')
u_prod_f.head()

,user_id,product_id,u_p_orders_total,product_name
0,1,196,10,Soda
1,1,10258,9,Pistachios
2,1,10326,1,Organic Fuji Apples
3,1,12427,10,Original Beef Jerky
4,1,13032,3,Cinnamon Toast Crunch


In [10]:
product_counts = u_prod_f['product_name'].value_counts()

rare_products = product_counts[product_counts < 1500].index  #def rare product

common_products = u_prod_f[~u_prod_f['product_name'].isin(rare_products)]
common_products['product_name'].nunique() 

1661

In [11]:
# Creating User Product Df
user_product_df = common_products.pivot_table(index=['user_id'], columns=['product_name'], values="u_p_orders_total")
user_product_df.head()

product_name,0% Greek Strained Yogurt,1% Low Fat Milk,1% Lowfat Milk,100 Calorie Per Bag Popcorn,100% Apple Juice,100% Cranberry Juice,100% Grated Parmesan Cheese,100% Lactose Free Fat Free Milk,100% Lemon Juice,100% Lime Juice,...,YoKids Strawberry Banana/Strawberry Yogurt,Yobaby Organic Plain Yogurt,Yoghurt Blueberry,"Yogurt, Lowfat, Strawberry","Yogurt, Strained Low-Fat, Coconut",Yukon Gold Potatoes 5lb Bag,ZBar Organic Chocolate Brownie Energy Snack,Zen Tea,Zero Calorie Cola,smartwater® Electrolyte Enhanced Water
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
user_product_df.shape

(205339, 1661)

In [13]:
df_train = pd.merge(order_products_train, orders, on='order_id', how='inner')
df_train = pd.merge(df_train, products, on='product_id', how='left')

dict_agg = {
            #'add_to_cart_order': ['mean'],
            #'days_since_prior_order': ['mean'],
            'reordered': ['count']
            #'order_number': ['max']
            }
u_prod_f_train = df_train.groupby(['user_id', 'product_id']).agg(dict_agg)
u_prod_f_train.columns = ['u_p_orders_total']
u_prod_f_train.reset_index(inplace=True)
u_prod_f_train = pd.merge(u_prod_f_train, products[['product_id', 'product_name']], on='product_id', how='left')
u_prod_f_train.head()

,user_id,product_id,u_p_orders_total,product_name
0,1,196,1,Soda
1,1,10258,1,Pistachios
2,1,13032,1,Cinnamon Toast Crunch
3,1,25133,1,Organic String Cheese
4,1,26088,1,Aged White Cheddar Popcorn


In [15]:
def compute_filtered_item_similarity(user_product_df, min_common_users, min_product_popularity):
    """
    Build item-item similarity matrix, keep similarities if items have at least min_common_users
    Skip items bought by fewer than min_product_popularity users
    """
    # Calculate how many users bought each product
    product_popularity = user_product_df.notna().sum()
    popular_items = product_popularity[product_popularity >= min_product_popularity].index.tolist()
    
    filtered_corr = pd.DataFrame(index=popular_items, columns=popular_items, dtype=float)

    for i, item_i in enumerate(tqdm(popular_items, desc="Computing item similarities")):
        vec_i = user_product_df[item_i]

        for j in range(i, len(popular_items)):
            item_j = popular_items[j]
            vec_j = user_product_df[item_j]

            mask = vec_i.notna() & vec_j.notna()
            common_users = mask.sum()

            if common_users >= min_common_users:
                x = vec_i[mask].values
                y = vec_j[mask].values
                if np.std(x) > 0 and np.std(y) > 0:
                    corr = np.corrcoef(x, y)[0, 1]
                    filtered_corr.at[item_i, item_j] = corr
                    filtered_corr.at[item_j, item_i] = corr

    return filtered_corr

In [16]:
item_similarity_matrix = compute_filtered_item_similarity(
    user_product_df,
    min_common_users=50,
    min_product_popularity=1000  # Skip rare products that only been bought 1000 times
)

Computing item similarities: 100%|█| 1661/1661 [52:04<00:00,  1


In [17]:
class ItemBasedRecommender:
    def __init__(self, user_product_df, similarity_matrix):
        self.user_product_df = user_product_df.fillna(0)
        self.similarity_matrix = similarity_matrix

    def recommend_products(self, user_id, n=10):
        user_vector = self.user_product_df.loc[user_id]
        scores = self.similarity_matrix.dot(user_vector).sort_values(ascending=False)

        purchased = set(self.user_product_df.loc[user_id][self.user_product_df.loc[user_id] > 0].index)
        scores = scores.drop(labels=purchased, errors='ignore')
        
        return scores.head(n).index.tolist()


In [18]:
user_gt_dict = u_prod_f_train.groupby("user_id")["product_name"].apply(set).to_dict()

In [20]:
class RecommenderEvaluator:
    def __init__(self, ground_truth, recommendations):
        self.ground_truth = ground_truth
        self.recommendations = recommendations

    def mean_precision_at_k(self, k=10):
        precisions = []
        for user, recs in self.recommendations.items():
            gt = self.ground_truth.get(user, set())
            hit_count = sum([1 for item in recs[:k] if item in gt])
            precisions.append(hit_count / k)
        return sum(precisions) / len(precisions)

    def mean_avg_precision(self):
        aps = []
        for user, recs in self.recommendations.items():
            gt = self.ground_truth.get(user, set())
            hits, score = 0, 0
            for i, item in enumerate(recs):
                if item in gt:
                    hits += 1
                    score += hits / (i + 1)
            if gt:
                aps.append(score / len(gt))
        return sum(aps) / len(aps)

    def mean_auc(self, N=10):
        aucs = []
        for user, recs in self.recommendations.items():
            gt = self.ground_truth.get(user, set())
            y_true = [1 if item in gt else 0 for item in recs]
            y_score = [1 / (i + 1) for i in range(len(recs))]
            if len(set(y_true)) == 2:
                auc = roc_auc_score(y_true, y_score)
                aucs.append(auc)
        return sum(aucs) / len(aucs)


In [21]:
# Get valid users 
valid_users = list(set(user_product_df.index).intersection(user_gt_dict.keys()))

# Initialize the recommender
recommender = ItemBasedRecommender(user_product_df, item_similarity_matrix)

# Generate recommendations
sample_users = valid_users[:10]  # take 10
recommendations = {
    user_id: recommender.recommend_products(user_id, n=10)
    for user_id in sample_users
}

# Evaluate recommendations
evaluator = RecommenderEvaluator(user_gt_dict, recommendations)
k_values = [5, 10, 20]
results = {}

for k in k_values:
    results[f"Precision@{k}"] = evaluator.mean_precision_at_k(k=k)

# Add MAP as a fixed metric (doesn't depend on cutoff)
results["MAP"] = evaluator.mean_avg_precision()

print(results)

{'Precision@5': 0.04, 'Precision@10': 0.02, 'Precision@20': 0.01, 'MAP': 0.01827956989247312}
